In [13]:
# Importing the necessary libraries and modules

import cantera as ct
import numpy as np

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [14]:
# Creating a fuel stream using the fuel information
# Fuel information by Altafini et al. (2003)
# https://doi.org/10.1016/S0196-8904(03)00025-6

ultimate = [52.00, 6.07, 41.55, 0.28, 0, 0]
ashDB = 0.10 / 100 # 0.1% ash, dry basis
HHV = 20.407 # MJ/kg
moistWB = 0.11 # 11% moisture, wet basis
moistDB = moistWB / (1 - moistWB)
fuelDryMass = 11.7 # kg
# Using the create_fuel_stream function to create the fuel
fuel = fs.create_fuel_stream(fuelDryMass, ultimate, ashDB, moistDB, HHV)
fuelMass = fuel.get_mass() # Fuel mass, in kg, including moisture

# Creating an air stream using an Air/Fuel ratio
AFR = 1.829 # Air/Fuel ratio, in kg/kg
airMass = AFR * fuelMass # Air mass, in kg
# Using the create_sir_stream function to create the air given a mass in kg
air = fs.create_air_stream(airMass)

# Simulating the gasification and saving the outlet results
# T = 832°C (isothermal)
# P = 0.93 bar

outlet = gs.gasify_isot(fuel, air, T=832+273.15, P=0.93*1E5, charFormation=0.1, directMethaneConv=0.2)

In [15]:
# The global mass balance will be the following equation:
# mass_in - mass_out = 0
# fuel_mass + air_mass - outlet_mass = 0
# Therefore, if the variable "global_mass_balance" is defined as follows, then it must be as close to zero as possible
global_mass_balance = fuelMass + airMass - outlet.get_mass()
# A simple test to check if the "global_mass_balance" is, in fact, zero (or as close as possible, up to 1e-8)
print(f'Global mass balance: {np.isclose(global_mass_balance, 0, atol=1e-8)}')

Global mass balance: True


In [16]:
# The following elements are present in the phases (solid and gas):
elements = ['C', 'H', 'O', 'N', 'S', 'Cl', 'Si', 'Ca', 'Al', 'Fe', 'Na', 'K', 'Mg', 'P', 'Ti', 'Cr', 'Ar']

for i, e in enumerate(elements):
    # For each element, the mole balance must be preserved.
    # Therefore, the "mole_balance" variable must be as close to zero as possible.
    mole_balance = fuel.element_moles(e) + air.element_moles(e) - outlet.element_moles(e)
    # A simple test to check if the "mole_balance" is, in fact, zero (or as close as possible, up to 1e-8)
    print(f'{e} mole balance: {np.isclose(mole_balance, 0, atol=1e-8)}')

C mole balance: True
H mole balance: True
O mole balance: True
N mole balance: True
S mole balance: True
Cl mole balance: True
Si mole balance: True
Ca mole balance: True
Al mole balance: True
Fe mole balance: True
Na mole balance: True
K mole balance: True
Mg mole balance: True
P mole balance: True
Ti mole balance: True
Cr mole balance: True
Ar mole balance: True
